کتابخانه های مورد نیاز

دانلود دیتاست مورد استفاده:   https://www.mediafire.com/file/j03djpy140rno4d/data.rar/file

In [40]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau


وارد کردن دیتاست

In [41]:
data_dir = "data"  
categories = os.listdir(data_dir) 

داده‌ها و برچسب‌ها 

In [42]:
images = []
labels = []

فرمت‌های تصویری پشتیبانی‌شده

In [43]:
valid_formats = ('.jpg', '.png', '.gif', '.jpeg')

تغییر سایز

In [44]:
img_size = 32

پیش‌ پردازش تصاویر

In [45]:
for label, category in enumerate(categories):  
    category_path = os.path.join(data_dir, category)  
    for file in os.listdir(category_path):  
        if file.lower().endswith(valid_formats):  
            file_path = os.path.join(category_path, file)  
            try:  
                
                img = cv2.imread(file_path, cv2.IMREAD_COLOR)  
                if img is not None:  
                    img_resized = cv2.resize(img, (img_size, img_size))  
                    images.append(img_resized)  
                    labels.append(label)  
                else:  
                    print(f"Error loading image: {file_path}")  
            except Exception as e:  
                print(f"Error processing {file_path}: {e}")  


Error loading image: data\Sad\crying-sad.gif


 و نرمال سازی تبدیل به آرایه numpy 

In [46]:
images = np.array(images) / 255.0  
labels = np.array(labels)

In [47]:
labels = to_categorical(labels, num_classes=len(categories))

ورود لیبل ها به صورت one hot

In [48]:
labels = to_categorical(labels, num_classes=len(categories))

تقسیم داده‌های آموزشی و آزمایشی

In [55]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)  


تغییر شکل تصاویر برای ورود به RNN

In [56]:
X_train = X_train.reshape(X_train.shape[0], 1, img_size * img_size * 3)   
X_test = X_test.reshape(X_test.shape[0], 1, img_size * img_size * 3) 

ساخت مدل RNN

In [57]:
model = Sequential()  
model.add(LSTM(128, input_shape=(1, img_size * img_size * 3), return_sequences=False))  
model.add(Dropout(0.2))  
model.add(Dense(64, activation='relu'))  
model.add(Dropout(0.2))  
model.add(Dense(len(categories), activation='softmax'))  

کامپایل مدل

In [58]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  


آموزش مدل

In [59]:
lr_scheduler = ReduceLROnPlateau(  
    monitor='val_loss', factor=0.5, patience=5, verbose=1  
)  
 
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=40, batch_size=32, callbacks=[lr_scheduler])

Epoch 1/40


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None, 3, 3), output.shape=(None, 3)

ارزیابی مدل

In [60]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None, 3, 3), output.shape=(None, 3)